# Quadratic program

A quadratic program is an optimization problem with a quadratic objective and affine equality and inequality constraints. 

A common standard form is the following:

$$  
    \begin{array}{ll}
    \text{minimize}   & (1/2)x^TPx + q^Tx\\
    \text{subject to} & Gx \leq h \\
                      & Ax = b.
    \end{array}
$$

Here $P \in \mathcal{S}^{n}_+$, $q \in \mathcal{R}^n$, $G \in \mathcal{R}^{m \times n}$, $h \in \mathcal{R}^m$, $A \in \mathcal{R}^{p \times n}$, and $b \in \mathcal{R}^p$ are problem data and $x \in \mathcal{R}^{n}$ is the optimization variable. The inequality constraint $Gx \leq h$ is elementwise.

A simple example of a quadratic program arises in finance. Suppose we have $n$ different 
stocks, an estimate $r \in \mathcal{R}^n$ of the expected return on each stock, and an 
estimate $\Sigma \in \mathcal{S}^{n}_+$ of the covariance of the returns. Then we solve 
the optimization problem:

$$  
    \begin{array}{ll}
    \text{minimize}   & (1/2)x^T\Sigma x - r^Tx\\
    \text{subject to} & x \geq 0 \\
                      & \mathbf{1}^Tx = 1,
    \end{array}
$$

to find a nonnegative portfolio allocation $x \in \mathcal{R}^n_+$ that optimally 
balances expected return and variance of return.

When we solve a quadratic program, in addition to a solution $x^\star$, we obtain a dual 
solution $\lambda^\star$ corresponding to the inequality constraints. A positive entry 
$\lambda^\star_i$ indicates that the constraint $g_i^Tx \leq h_i$ holds with equality 
for $x^\star$ and suggests that changing $h_i$ would change the optimal value.

## Quadratic Program Example

In the following code, we solve a quadratic program with CVXPY.

In [1]:
# Import packages.
import cvxpy as cp
import numpy as np

rng = np.random.default_rng(1)

# generate a random feasible quadratic program.
m = 15
n = 10
p = 5
P = rng.standard_normal((n, n))
P = P.T @ P
q = rng.standard_normal(n)
G = rng.standard_normal((m, n))

# generate a feasible point first, then construct constraints around it
x_feasible = rng.standard_normal(n)
h = G @ x_feasible + rng.exponential(1, m)

A = rng.standard_normal((p, n))
b = A @ x_feasible

# define and solve the CVXPY problem.
x = cp.Variable(n)
prob = cp.Problem(
    cp.Minimize((1 / 2) * cp.quad_form(x, P) + q.T @ x), [G @ x <= h, A @ x == b]
)
out = prob.solve(solver=cp.SCS, verbose=True)

# Print result.
print("The solver status is", prob.status)
print("\nThe optimal value is", prob.value)
print("A solution x is")
print(x.value)
print("A dual solution corresponding to the inequality constraints is")
print(prob.constraints[0].dual_value)

(CVXPY) Aug 07 04:22:13 PM: Your problem has 10 variables, 20 constraints, and 0 parameters.


(CVXPY) Aug 07 04:22:13 PM: It is compliant with the following grammars: DCP, DQCP


(CVXPY) Aug 07 04:22:13 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)


(CVXPY) Aug 07 04:22:13 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.


(CVXPY) Aug 07 04:22:13 PM: Your problem is compiled with the CPP canonicalization backend.


...

/mnt/c/Users/lange/OneDrive/Projects/2025/cvx/cvx-package-template/.venv/lib/python3.13/site-packages/scs/__init__.py:83: UserWarning: Converting A to a CSC (compressed sparse column) matrix; may take a while.
  warn(
/mnt/c/Users/lange/OneDrive/Projects/2025/cvx/cvx-package-template/.venv/lib/python3.13/site-packages/scs/__init__.py:113: UserWarning: Converting P to a CSC (compressed sparse column) matrix; may take a while.
  warn(
(CVXPY) Aug 07 04:22:13 PM: Problem status: optimal


(CVXPY) Aug 07 04:22:13 PM: Optimal value: 1.678e+01


(CVXPY) Aug 07 04:22:13 PM: Compilation took 6.976e-02 seconds


(CVXPY) Aug 07 04:22:13 PM: Solver (including time spent in interface) took 4.815e-03 seconds


                                     CVXPY                                     
                                     v1.7.1                                    
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
------------------------------------------------------------------
	       SCS v3.2.7 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 10, constraints m: 20
cones: 	  z: primal zero / dual free vars: 5
	  l: linear vars: 15
settings: eps_abs: 

In [2]:
assert prob.status == cp.OPTIMAL, "Solver result not optimal"
assert prob.value is not None, "Problem value is None"